In [1]:
from pymongo import MongoClient
# import getpass
import json
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
password = os.getenv("ATLAS_MONGO_PASS")

In [4]:
# password = getpass.getpass("Insert your AtlasMongoDB password: ")
connection = "mongodb+srv://masdevallia:{}@clusterdataanalysis-9nklb.mongodb.net/test?retryWrites=true&w=majority".format(password)

In [5]:
client = MongoClient(connection)

In [6]:
client

MongoClient(host=['clusterdataanalysis-shard-00-00-9nklb.mongodb.net:27017', 'clusterdataanalysis-shard-00-01-9nklb.mongodb.net:27017', 'clusterdataanalysis-shard-00-02-9nklb.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='ClusterDataAnalysis-shard-0', ssl=True)

In [8]:
def connectCollection(database, collection):
    db = client[database]
    coll = db[collection]
    return db, coll

In [5]:
import pandas as pd
df = pd.read_json('./input/chats.json')
df.head(3)

,datetime,idChat,idMessage,idUser,text,userName
0,2019-10-17 10:15:41,0,0,0,"Hey Mike, whats up??",John Wick
1,2019-10-17 10:16:59,0,1,1,Dude!!! ðŸ˜€ï¸� Did you watch the game last ni...,Mike Wazowski
2,2019-10-17 10:17:31,0,2,0,"No, had to work. How was it?",John Wick


In [8]:
db, coll = connectCollection('chats','messages')
with open('./input/chats.json', encoding="utf8") as f:
    chats_json = json.load(f)
coll.insert_many(chats_json)

In [12]:
len(set(df['idUser'])) # 5 chats, 50 mensajes, 8 users

8

In [ ]:
userDF = df[['idUser','userName']]
userDF.drop_duplicates(inplace=True)

In [24]:
userDF

,idUser,userName
0,0,John Wick
1,1,Mike Wazowski
10,2,Leia Organa
11,3,Tony Stark
21,4,Ellen Ripley
30,5,J. Jonah Jameson
40,6,Danny Torrances Finger
41,7,Danny Torrance


In [25]:
# Importing cleaned data to MongoDB:
coll2 = db['users']
coll2.insert_many(userDF.to_dict('records'))

In [26]:
userDF.to_json('./input/users.json', orient="records")

In [9]:
db, coll = connectCollection('chats','users')

In [10]:
data = list(coll.find({}))

In [11]:
data

[{'_id': ObjectId('5de160c2b6cad550d4929fa9'),
  'idUser': 0,
  'userName': 'John Wick'},
 {'_id': ObjectId('5de160c2b6cad550d4929faa'),
  'idUser': 1,
  'userName': 'Mike Wazowski'},
 {'_id': ObjectId('5de160c2b6cad550d4929fab'),
  'idUser': 2,
  'userName': 'Leia Organa'},
 {'_id': ObjectId('5de160c2b6cad550d4929fac'),
  'idUser': 3,
  'userName': 'Tony Stark'},
 {'_id': ObjectId('5de160c2b6cad550d4929fad'),
  'idUser': 4,
  'userName': 'Ellen Ripley'},
 {'_id': ObjectId('5de160c2b6cad550d4929fae'),
  'idUser': 5,
  'userName': 'J. Jonah Jameson'},
 {'_id': ObjectId('5de160c2b6cad550d4929faf'),
  'idUser': 6,
  'userName': 'Danny Torrances Finger'},
 {'_id': ObjectId('5de160c2b6cad550d4929fb0'),
  'idUser': 7,
  'userName': 'Danny Torrance'}]

In [13]:
user_id_list = []
for i in range(len(df)):
    for e in data:
        if df.at[i,'idUser'] == e['idUser']:
            user_id_list.append(e['_id'])

In [18]:
df['user_id'] = user_id_list

In [26]:
coll3 = db['messages_linked']
coll3.insert_many(df.to_dict('records'))